# Определение PartOfSpeech (частей речи) с помощью свёрточных нейросетей


## Импорт библиотек

In [2]:

from sklearn.metrics import classification_report

import numpy as np

import wget
import pyconll
import re
import collections

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset


Новая библиотека для загрузки корпуса Pyconll

Датасет размеченный для распознавания PoS - SyntaGRus. Также может применяться для синтаксичего и морфологическего разбора

## Загружаем датасет 

In [2]:
!wget -O ../dataset/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ../dataset/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2024-03-03 15:51:23--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736599 (39M) [text/plain]
Сохранение в: ‘../dataset/ru_syntagrus-ud-train.conllu’

../dataset/ru_synta 100%[===================>]  38,85M  2,62MB/s    за 18s     

2024-03-03 15:51:41 (2,19 MB/s) - ‘../dataset/ru_syntagrus-ud-train.conllu’ сохранён [40736599/40736599]

--2024-03-03 15:51:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.

In [3]:
train_data = pyconll.load_from_file('../dataset/ru_syntagrus-ud-train.conllu')
test_data = pyconll.load_from_file('../dataset/ru_syntagrus-ud-dev.conllu')

Вид разметки

In [4]:
for sent in train_data[:1]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT



In [5]:
MAX_SENT_LEN = max(len(sent) for sent in train_data)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in train_data for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [6]:
all_train_texts = [' '.join(token.form for token in sent) for sent in train_data]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

## Построение словаря символов

### Методы, написанные разработчиками курса. Токенизатор, и Сборщик словаря символов

In [7]:
TOKEN_RE = re.compile(r'[\w\d]+') 

def tokenize_text_simple_regex(txt, min_token_size=4):
    #print(txt)
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

def character_tokenize(txt): 
    return list(txt)

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs): 
    #print(texts)
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

In [8]:
# Метод, написанный преподавателями курса
def build_vocabulary(tokenized_texts, max_size=50000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq


PAD_TOKEN = '__PAD__'
NUMERIC_TOKEN = '__NUMBER__'
NUMERIC_RE = re.compile(r'^([0-9.,e+\-]+|[mcxvi]+)$', re.I)

### Формирование словаря символов

In [9]:
train_simvols_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
simvols_vocab, word_doc_freq = build_vocabulary(train_simvols_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(simvols_vocab))
print(list(simvols_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


### Нумерация частей речи

In [10]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in train_data for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

### Преобразование корпуса данных в Тензор, для pytorch

In [11]:
import torch
from torch.utils.data import TensorDataset

# На вход получает список токенизированных предложений+Pos, список нумерованных символов, 
def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
    inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long) 
    # +2 отступ для понимания расположения позиции символа
    targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

    for sent_i, sent in enumerate(sentences): ## итерация по предложениям
        for token_i, token in enumerate(sent): ## итерация по токенам
            if token.form is None:
                continue
            targets[sent_i, token_i] = label2id.get(token.upos, 0)
            for char_i, char in enumerate(token.form):
                inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)                
                            
    return inputs, targets

In [12]:
train_inputs, train_labels = pos_corpus_to_tensor(train_data, simvols_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(test_data, simvols_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [13]:
train_inputs[1][:2] # Первые два слова во втором преддожении

tensor([[ 0, 38,  4, 25,  4, 11, 19,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [17]:
train_labels[1] # Целевое значение - Частей речи. Лишние ноли необходимы для выравнивания матриц, и распараллеливания

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## Инициализация сверточной архитектуры PosTagger по примеру из семинара

### Вспомогательная свёрточная cтруктура

In [18]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                # размерность сверточного слоя -# Одномерная свёртка
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                # Отключение "перегруженных нейронов" для равномерной нагрузки
                nn.Dropout(dropout),
                # Функция активации
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    # Функция реализующая алгоритм. 
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

###  Cвёрточная cтруктура на уровне отдельных токенов

In [16]:
### Предсказание части речи на уровне отдельных токенов.

class SingleTokenPOSTagger(nn.Module):    
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

## Применение PosTagger на уровне токенов

In [32]:
single_token_model = SingleTokenPOSTagger(len(simvols_vocab), len(label2id),
                                            embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.prod(t.shape) for t in single_token_model.parameters()))


Количество параметров 47314


### Методы из библиотек курса

In [19]:
import copy
import datetime
import random
import traceback
import numpy as np
import torch
from torch.utils.data import DataLoader

def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)

    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
        try:
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                           (datetime.datetime.now() - epoch_start).total_seconds()))
            print('Среднее значение функции потерь на обучении', mean_train_loss)



            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model


def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []

    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    labels = []
    with torch.no_grad():
        import tqdm
        for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
            batch_x = copy_data_to_device(batch_x, device)

            if return_labels:
                labels.append(batch_y.numpy())

            batch_pred = model(batch_x)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)
        
def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))

### Обучение сверточной модели на уровне токенов

In [33]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=3,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=100,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                           factor=0.5,verbose=True))

/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Эпоха 0
Эпоха: 101 итераций, 155.47 сек
Среднее значение функции потерь на обучении 0.2384020203337221
Среднее значение функции потерь на валидации 0.07113874028667365
Новая лучшая модель!

Эпоха 1
Эпоха: 101 итераций, 153.16 сек
Среднее значение функции потерь на обучении 0.05273136068688761
Среднее значение функции потерь на валидации 0.04974862341169674
Новая лучшая модель!

Эпоха 2
Эпоха: 101 итераций, 155.23 сек
Среднее значение функции потерь на обучении 0.04166504187454091
Среднее значение функции потерь на валидации 0.04457950355983017
Новая лучшая модель!



In [37]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [03:46,  3.39it/s]                                                        
/tmp/ipykernel_4638/2011350103.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels))


Среднее значение функции потерь на обучении 0.024675922468304634
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.86      0.91      0.89     43357
         ADP       1.00      0.98      0.99     39344
         ADV       0.85      0.84      0.85     22733
         AUX       0.82      0.79      0.80      3537
       CCONJ       0.88      0.98      0.93     15168
         DET       0.71      0.91      0.80     10781
        INTJ       1.00      0.08      0.15        50
        NOUN       0.97      0.88      0.92    103538
         NUM       0.93      0.87      0.90      5640
        PART       0.97      0.76      0.85     13556
        PRON       0.96      0.72      0.82     18734
       PROPN       0.83      0.89      0.86     14854
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.78      0.95      0.85      8057
         SYM       1.00      0.99      0.99       420
        VERB    

279it [01:23,  3.32it/s]                                                        
/tmp/ipykernel_4638/2011350103.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels))


Среднее значение функции потерь на валидации 0.028754806146025658
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.83      0.90      0.86     15103
         ADP       1.00      0.97      0.99     13717
         ADV       0.81      0.81      0.81      7783
         AUX       0.82      0.75      0.78      1390
       CCONJ       0.89      0.98      0.93      5672
         DET       0.71      0.86      0.78      4265
        INTJ       1.00      0.04      0.08        24
        NOUN       0.95      0.87      0.91     36238
         NUM       0.84      0.81      0.83      1734
        PART       0.96      0.73      0.83      5125
        PRON       0.95      0.73      0.83      7444
       PROPN       0.80      0.84      0.82      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.76      0.95      0.84      2865
         SYM       1.00      0.85      0.92        62
        VERB   

## Применение PosTagger на уровне предложения

In [20]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits


### Обучение сверточной модели на уровне предложения

In [21]:
sentence_level_model = SentenceLevelPOSTagger(len(simvols_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.prod(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84370


In [22]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=3,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=100,
                                              max_batches_per_epoch_val=100,
                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                           factor=0.5,verbose=True))

/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Эпоха 0
Эпоха: 101 итераций, 155.58 сек
Среднее значение функции потерь на обучении 0.19894202418699122
Среднее значение функции потерь на валидации 0.06185003311032116
Новая лучшая модель!

Эпоха 1
Эпоха: 101 итераций, 154.14 сек
Среднее значение функции потерь на обучении 0.05146126087644313
Среднее значение функции потерь на валидации 0.04147836896083733
Новая лучшая модель!

Эпоха 2
Эпоха: 101 итераций, 154.14 сек
Среднее значение функции потерь на обучении 0.03901225387459934
Среднее значение функции потерь на валидации 0.031702461717004825
Новая лучшая модель!



In [23]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [03:37,  3.52it/s]                                                        
/tmp/ipykernel_3181/4087871673.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels))


Среднее значение функции потерь на обучении 0.029104648157954216


/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Pre

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.82      0.89      0.85     43357
         ADP       1.00      0.97      0.98     39344
         ADV       0.80      0.70      0.75     22733
         AUX       0.84      0.91      0.87      3537
       CCONJ       0.88      0.98      0.93     15168
         DET       0.84      0.83      0.83     10781
        INTJ       0.00      0.00      0.00        50
        NOUN       0.92      0.90      0.91    103538
         NUM       0.90      0.73      0.81      5640
        PART       0.96      0.72      0.83     13556
        PRON       0.90      0.85      0.87     18734
       PROPN       0.81      0.89      0.85     14854
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.84      0.86      0.85      8057
         SYM       1.00      0.95      0.97       420
        VERB       0.83      0.91      0.87     47731
           X       0.00    

279it [01:18,  3.54it/s]                                                        
/tmp/ipykernel_3181/4087871673.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels))


Среднее значение функции потерь на валидации 0.031789667904376984


/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.79      0.87      0.83     15103
         ADP       0.99      0.97      0.98     13717
         ADV       0.76      0.67      0.71      7783
         AUX       0.84      0.89      0.86      1390
       CCONJ       0.89      0.98      0.93      5672
         DET       0.83      0.78      0.80      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.90      0.89      0.89     36238
         NUM       0.81      0.63      0.71      1734
        PART       0.96      0.71      0.82      5125
        PRON       0.89      0.83      0.86      7444
       PROPN       0.80      0.87      0.83      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.82      0.85      0.84      2865
         SYM       1.00      0.69      0.82        62
        VERB       0.82      0.90      0.86     17110
           X       0.00    

/home/michalik/github/course_work/stepik_coursework_TextProcessing/Seminar_1_Classification/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Класс POSTagger из библиотеки курса 

In [26]:
class POSTagger:
    def __init__(self, model, char2id, id2label, max_sent_len, max_token_len):
        self.model = model
        self.char2id = char2id
        self.id2label = id2label
        self.max_sent_len = max_sent_len
        self.max_token_len = max_token_len

    def __call__(self, sentences):
        tokenized_corpus = tokenize_corpus(sentences, min_token_size=1)

        inputs = torch.zeros((len(sentences), self.max_sent_len, self.max_token_len + 2), dtype=torch.long)

        for sent_i, sentence in enumerate(tokenized_corpus):
            for token_i, token in enumerate(sentence):
                for char_i, char in enumerate(token):
                    inputs[sent_i, token_i, char_i + 1] = self.char2id.get(char, 0)

        dataset = TensorDataset(inputs, torch.zeros(len(sentences)))
        predicted_probs = predict_with_model(self.model, dataset)  # SentenceN x TagsN x MaxSentLen
        predicted_classes = predicted_probs.argmax(1)

        result = []
        for sent_i, sent in enumerate(tokenized_corpus):
            result.append([self.id2label[cls] for cls in predicted_classes[sent_i, :len(sent)]])
        return result

## Применение полученных теггеров и сравнение

In [34]:
single_token_pos_tagger = POSTagger(single_token_model, simvols_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, simvols_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [35]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [36]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 13.95it/s]                                                          

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-ADJ

глокая-ADJ куздра-NOUN штеко-DET будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-NOUN три-NUM будет-VERB дырка-NOUN

три-NUM да-NOUN три-NUM будет-VERB шесть-VERB

сорок-NOUN сорок-NOUN



In [37]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 15.47it/s]                                                          

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-ADJ

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-NOUN три-NUM будет-AUX дырка-NOUN

три-NUM да-NOUN три-NUM будет-AUX шесть-VERB

сорок-NOUN сорок-NOUN



# Заключение

Частеричная омономия осталась в той же форме и не была решена в полной мере. 

Однако величина метрики F1-Score на валидации, кроме особых случаев, вполне удовлетворительно и составляет
для 
- Сверточной модели на уровне токенов = 81%
- Сверточной модели на уровне предложений (бóльший учёт контекста) = 76%